In [1]:
from neuron import h,gui2
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox,VBox,Label,Layout
h.load_file('stdrun.hoc')
gui2.set_backend('jupyter')

In [2]:
h.load_file('pyr3_template.hoc')
Cell = h.pyr3(0)
h.psection(Cell)

	1 

-----

Layer 2/3 Cortical Neurons Model based on Traub RD (2003)

-----
pyr3[0].comp[1] { nseg=1  L=15  Ra=250
	/*location 0 attached to cell 0*/
	/* First segment only */
	insert morphology { diam=16}
	insert capacitance { cm=0.9}
	insert pas { g_pas=2e-05 e_pas=-70}
	insert IClamp { del=0 dur=150 amp=-0.15}
	insert cad { phi_cad=26000 beta_cad=0.01}
	insert naf { gbar_naf=0.1875}
	insert nap { gbar_nap=0.00042 vhalf_nap=-48 slope_nap=10}
	insert na_ion { ena=50}
	insert kdr { gbar_kdr=0.125}
	insert ka { gbar_ka=0.03}
	insert kc { gbar_kc=0.012 slope_kc=6 vhalf_kc=-19}
	insert kahp { gbar_kahp=0.0001}
	insert k2 { gbar_k2=0.0001}
	insert km { gbar_km=0.0075}
	insert k_ion { ek=-95}
	insert cat { gbar_cat=0.0001}
	insert cal { gbar_cal=0.0005 vhalf_cal=-20 slope_cal=8}
	insert ca_ion {}
	insert ar { gbar_ar=0.00025}
}


1.0

In [3]:
h.v_init = -70
h.tstop = 450
h.dt = 0.025
v0_vec = h.Vector()
t_vec = h.Vector()
v0_vec.record(Cell.comp[1](0.5)._ref_v)
t_vec.record(h._ref_t)
ina = h.Vector()
ik = h.Vector()
ical = h.Vector()
icat = h.Vector()
minf_nap = h.Vector()
minf_cal = h.Vector()
minf_kc = h.Vector()

ina.record(Cell.comp[1](0.5)._ref_ina_nap)
ik.record(Cell.comp[1](0.5)._ref_ik_kc)
ical.record(Cell.comp[1](0.5)._ref_ica_cal)
icat.record(Cell.comp[1](0.5)._ref_i_cat)

minf_nap.record(Cell.comp[1](0.5)._ref_minf_nap)
minf_cal.record(Cell.comp[1](0.5)._ref_minf_cal)
minf_kc.record(Cell.comp[1](0.5)._ref_minf_kc)


1.0

In [4]:
def Bursting (amp,dnap,dkc,vhalfnap,vhalfkca,vhalfcal,slopenap,slopekca,slopecal):

    for sec in Cell.Soma:
        gbar_naf1  = 150e-3 * 1.25
        sec.gbar_nap  = dnap * 0.0032 * gbar_naf1
        sec.gbar_kc   = dkc * 7.5e-3
        sec.vhalf_nap = vhalfnap
        sec.vhalf_kc = vhalfkca
        sec.vhalf_cal = vhalfcal
        sec.slope_nap = slopenap
        sec.slope_kc = slopekca
        sec.slope_cal = slopecal
    for sec in Cell.Dendrites:
        gbar_naf2  = 6.25e-3 
        sec.gbar_nap  = dnap * 0.0032 * gbar_naf2
        sec.vhalf_nap = vhalfnap
        sec.vhalf_cal = vhalfcal
        sec.slope_nap = slopenap
        sec.slope_cal = slopecal
    for sec in Cell.Prox:
        gbar_naf3  = 75e-3 * 1.25
        sec.gbar_nap  = dnap * 0.0032 * gbar_naf3
        sec.gbar_kc   = dkc * 7.5e-3
        sec.vhalf_nap = vhalfnap
        sec.vhalf_kc = vhalfkca
        sec.slope_nap = slopenap
        sec.slope_kc = slopekca
    for sec in Cell.Dist:
        sec.slope_cal = slopecal
        sec.vhalf_cal = vhalfcal
    gbar_naf4  = 125e-3
    Cell.comp[38].gbar_nap  = dnap * 0.0032 * gbar_naf4
    Cell.comp[38].gbar_kc   = dkc * 7.5e-3
    Cell.comp[38].vhalf_nap = vhalfnap
    Cell.comp[38].vhalf_kc = vhalfkca
    Cell.comp[38].slope_nap = slopenap
    Cell.comp[38].slope_kc = slopekca

    stim = h.IClamp(Cell.comp[1](0.5)) 
    stim.delay = 150 
    stim.dur = 250  
    stim.amp = amp
    h.run()
    #plt.figure(figsize=(10,16))
    #plt.subplot(4,1,1)
    plt.plot(t_vec, v0_vec)
    plt.xlim(100,450)
    plt.xlabel('time(ms)')
    plt.ylabel('mV')
    '''
    plt.subplot(4,1,2)
    plt.plot(v0_vec,minf_nap,'b')
    
    plt.xlabel('time(ms)')
    plt.ylabel('current(nA)')
    plt.subplot(4,1,3)
    plt.plot(v0_vec,minf_cal,'r')
  
    plt.xlabel('time(ms)')
    plt.ylabel('current(nA)')
    plt.subplot(4,1,4)
    plt.plot(v0_vec,minf_kc,'g')
   
    plt.xlabel('time(ms)')
    plt.ylabel('current(nA)')
    '''
    plt.show()

In [5]:
amp = 1.0
dnap = 1.0
dkc = 1.3
vhalfnap = -48
vhalfkca = -19
vhalfcal = -20
slopenap = 10
slopekca = 6
slopecal = 8

w_amp = widgets.FloatSlider(value=amp,min=0,max=3,step=0.1,continuous_update=False,readout_format='.1f')
w_dnap = widgets.FloatSlider(value=dnap,min=0,max=2,step=0.1,continuous_update=False,readout_format='.1f')
w_dkc = widgets.FloatSlider(value=dkc,min=0,max=2,step=0.1,continuous_update=False,readout_format='.1f')

w_halfnap = widgets.FloatSlider(value=vhalfnap,min=-64,max=-30,step=.1,continuous_update=False,readout_format='.1f')
w_halfkca = widgets.FloatSlider(value=vhalfkca,min=-30,max=0,step=.1,continuous_update=False,readout_format='.1f')
w_halfcal = widgets.FloatSlider(value=vhalfcal,min=-30,max=0,step=.1,continuous_update=False,readout_format='.1f')

w_slopenap = widgets.FloatSlider(value=slopenap,min=0,max=70,step=.1,continuous_update=False,readout_format='.1f')
w_slopekca = widgets.FloatSlider(value=slopekca,min=0,max=70,step=.1,continuous_update=False,readout_format='.1f')
w_slopecal = widgets.FloatSlider(value=slopecal,min=0,max=70,step=.1,continuous_update=False,readout_format='.1f')

ui =  HBox([VBox([Label('amp'),Label('dnap'),Label('dkc'),Label('halfnap'),Label('halfkca'),Label('halfcal'),Label('slopenap'),Label('slopekca'),Label('slopecal')],layout=Layout(width='10%')),
            VBox([w_amp,w_dnap,w_dkc,w_halfnap,w_halfkca,w_halfcal,w_slopenap,w_slopekca,w_slopecal],layout=Layout(width='38%'))])

out = widgets.interactive_output(Bursting,{'amp':w_amp,'dnap':w_dnap,'dkc':w_dkc,'vhalfnap':w_halfnap,'vhalfkca':w_halfkca,'vhalfcal':w_halfcal,'slopenap':w_slopenap,'slopekca':w_slopekca,'slopecal':w_slopecal})

display(ui,out)

Output()